In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [11]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.006372                    1  0.194546
0.000750        2 -0.204306                    2 -0.275375
0.001052        3 -0.144126                    3 -0.199134
0.001476        4 -0.114643                    4 -0.171477
0.002070        5 -0.090063                    5 -0.158840
0.002904        6 -0.066733                    6 -0.142954
0.004074        7 -0.044387                    7 -0.120778
0.005714        8 -0.023448                    8 -0.084861
0.008015        9 -0.004370                    9 -0.042313
0.011243       10  0.013043                   10  0.005856
0.015771       11  0.029747                   11  0.057942
0.022122       12  0.047905                   12  0.112113
0.031031       13  0.075561                   13  0.169551
0.043528       14  0.096618                   14  0.204089
0.061057       15  0.094248                   15  0.192009
0.085645       16  0.089566                   16  0.155655
0.120136       17  0.092757                   17  0.119583
0.168516       18  0.099813                   18  0.094253
0.236378       19  0.111086                   19  0.085193
0.331549       20  0.130938                   20  0.097093
0.465100       21  0.179570                   21  0.152578
0.652400       22  0.163299                   22  0.149473
0.915100       23  0.085471                   23  0.093515
1.283650       24  0.042146                   24  0.063241
1.800600       25  0.023025                   25  0.043148
2.525700       26  0.012075                   26  0.023621
3.542800       27  0.004840                   27  0.006421
4.969550       28  0.001632                   28 -0.002091
6.970850       29  0.001317                   29 -0.000155
9.778100       30  0.001079                   30  0.002748
13.715850      31  0.000427                   31  0.002466
19.239350      32 -0.000345                   32  0.000760
26.987250      33 -0.000397                   33 -0.000256
37.855300      34  0.000015                   34 -0.000131
53.100050      35  0.000124                   35 -0.000049
73.887500      36  0.000152                   36  0.000042
97.662500      37  0.000172                   37  0.000156
121.437500     38  0.000002                   38  0.000048
145.212500     39 -0.000216                   39 -0.000142
168.987500     40 -0.000347                   40 -0.000258
192.762500     41 -0.000471                   41 -0.000402
216.537500     42 -0.000604                   42 -0.000582
240.312500     43 -0.000759                   43 -0.000801
264.087500     44 -0.000997                   44 -0.001115
287.862500     45 -0.001514                   45 -0.001729
311.637500     46 -0.001867                   46 -0.002145
335.412500     47 -0.001637                   47 -0.001958
359.187500     48 -0.001302                   48 -0.001633
382.962500     49 -0.001113                   49 -0.001454
406.737500     50 -0.000903                   50 -0.001238
430.512500     51 -0.000644                   51 -0.000928
454.287500     52 -0.000321                   52 -0.000579
478.062500     53  0.000077                   53 -0.000107
501.837500     54  0.000555                   54  0.000324
525.612500     55  0.001120                   55  0.000759
549.387500     56  0.001763                   56  0.001236
573.162500     57  0.002474                   57  0.001822
596.937500     58  0.003229                   58  0.002557
620.712500     59  0.003999                   59  0.003291
644.487500     60  0.004748                   60  0.004237
668.262500     61  0.005453                   61  0.005047
692.037500     62  0.006099                   62  0.005806
715.812500     63  0.006690                   63  0.006492
739.587500     64  0.007249                   64  0.007125
763.362500     65  0.007817                   65  0.007752
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -1.140560  0.000000 -1.140560 -1.118276  9.129488e-09   
0.000624    2     -1.140581  0.000021 -1.140560 -1.118304  4.219682e-05   
0.000876    3     -1.140591  0.000025 -1.140566 -1.118319  4.920004e-05   
0.001229    4     -1.140606  0.000034 -1.140572 -1.118339  6.088376e-05   
0.001723    5     -1.140625  0.000046 -1.140579 -1.118365  7.712118e-05   
0.002417    6     -1.140650  0.000064 -1.140586 -1.118400  9.834021e-05   
0.003391    7     -1.140684  0.000090 -1.140594 -1.118444  1.261897e-04   
0.004757    8     -1.140727  0.000126 -1.140601 -1.118502  1.641769e-04   
0.006672    9     -1.140784  0.000178 -1.140606 -1.118575  2.187308e-04   
0.009359    10    -1.140858  0.000250 -1.140608 -1.118669  2.990434e-04   
0.013128    11    -1.140952  0.000350 -1.140602 -1.118787  4.193034e-04   
0.018415    12    -1.141070  0.000487 -1.140583 -1.118932  6.004317e-04   
0.025830    13    -1.141215  0.000674 -1.140541 -1.119104  8.715434e-04   
0.036232    14    -1.141377  0.000929 -1.140448 -1.119293  1.268924e-03   
0.050823    15    -1.141540  0.001259 -1.140281 -1.119482  1.810983e-03   
0.071291    16    -1.141713  0.001660 -1.140053 -1.119679  2.473746e-03   
0.100000    17    -1.141905  0.002157 -1.139748 -1.119893  3.216755e-03   
0.140271    18    -1.142108  0.002803 -1.139306 -1.120111  4.005528e-03   
0.196760    19    -1.142296  0.003658 -1.138638 -1.120308  4.833062e-03   
0.275997    20    -1.142406  0.004811 -1.137595 -1.120438  5.763455e-03   
0.387100    21    -1.142253  0.006382 -1.135872 -1.120354  6.957122e-03   
0.543100    22    -1.141046  0.008492 -1.132553 -1.119182  8.604353e-03   
0.761700    23    -1.138599  0.010274 -1.128325 -1.116749  1.004323e-02   
1.068500    24    -1.136025  0.010806 -1.125219 -1.114180  1.087239e-02   
1.498800    25    -1.133736  0.010666 -1.123070 -1.111822  1.173878e-02   
2.102400    26    -1.131709  0.010285 -1.121424 -1.109623  1.262490e-02   
2.949000    27    -1.129951  0.009739 -1.120213 -1.107603  1.297444e-02   
4.136600    28    -1.128581  0.009049 -1.119532 -1.105908  1.218269e-02   
5.802500    29    -1.127610  0.008400 -1.119210 -1.104632  1.049414e-02   
8.139200    30    -1.126838  0.007992 -1.118846 -1.103556  9.374644e-03   
11.417000   31    -1.126165  0.007739 -1.118426 -1.102571  9.457286e-03   
16.014700   32    -1.125692  0.007499 -1.118194 -1.101827  1.005561e-02   
22.464000   33    -1.125781  0.007323 -1.118457 -1.101769  1.057815e-02   
31.510500   34    -1.126580  0.007697 -1.118883 -1.102583  1.111775e-02   
44.200100   35    -1.127815  0.008955 -1.118860 -1.103879  1.221788e-02   
62.000000   36    -1.129455  0.010856 -1.118599 -1.105532  1.376784e-02   
85.775000   37    -1.131390  0.013221 -1.118170 -1.107390  1.574312e-02   
109.550000  38    -1.133035  0.015349 -1.117686 -1.108944  1.773758e-02   
133.325000  39    -1.134592  0.016913 -1.117679 -1.110433  1.936164e-02   
157.100000  40    -1.136370  0.018083 -1.118288 -1.112238  2.076513e-02   
180.875000  41    -1.138405  0.019140 -1.119265 -1.114411  2.221119e-02   
204.650000  42    -1.140734  0.020141 -1.120593 -1.117009  2.367835e-02   
228.425000  43    -1.143391  0.021097 -1.122294 -1.120105  2.513583e-02   
252.200000  44    -1.146452  0.022022 -1.124430 -1.123784  2.655872e-02   
275.975000  45    -1.150174  0.022935 -1.127239 -1.128309  2.794295e-02   
299.750000  46    -1.155505  0.024001 -1.131504 -1.134683  2.944766e-02   
323.525000  47    -1.163017  0.026255 -1.136762 -1.143449  3.217311e-02   
347.300000  48    -1.172036  0.030663 -1.141373 -1.153892  3.710054e-02   
371.075000  49    -1.181951  0.036912 -1.145039 -1.165323  4.393291e-02   
394.850000  50    -1.192986  0.044814 -1.148172 -1.177906  5.242181e-02   
418.625000  51    -1.205372  0.054656 -1.150716 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')